In [1]:
import polars as pl

In [2]:
dfOutputs = pl.read_csv("../data/outputs.csv")

In [3]:
import re

def isNujna(text):
    if re.search('Nujna prometna informacija', text, re.IGNORECASE):
        return 1
    if re.search('Nujne prometne informacije', text, re.IGNORECASE):
        return 1
    
    return 0

def isNova(text):
    if re.search('Nova prometna informacija', text, re.IGNORECASE):
        return 1
    if re.search('Nove prometne informacije', text, re.IGNORECASE):
        return 1
    
    return 0

In [4]:
# extract things like 1., 2. in 3. program

def extractPrograms(text):
    pattern = r'(\d+\.)\s*(?:in|,|&)?\s*(?=\d+\.|program)'

    matches = re.findall(pattern, text)

    m = []

    for match in matches:
        p = match.strip().replace(".", "")
        if int(p) <= 3:
            m.append(p)

    return ",".join(m)

In [5]:
dfOutputs = dfOutputs.with_columns([
    (pl.col("content").map_elements(lambda x: isNujna(x), return_dtype=pl.Int8).alias('nujna')),
    (pl.col("content").map_elements(lambda x: isNova(x), return_dtype=pl.Int8).alias('nova')),
    (pl.col("content").map_elements(lambda x: extractPrograms(x), return_dtype=pl.String).alias('programs')),
])

In [6]:
dfOutputs.head()

year,month,file_name,content,date,time,nujna,nova,programs
i64,str,str,str,str,str,i8,i8,str
2022,"""April""","""TMP-1""","""Prometne informacije 30.…","""2022-04-30""","""18:30""",0,0,"""1,2"""
2022,"""April""","""TMP-10""","""Prometne informacije 30.…","""2022-04-30""","""13:00""",0,0,"""1,3"""
2022,"""April""","""TMP-100""","""Prometne informacije 27.…","""2022-04-27""","""06:30""",0,0,"""1"""
2022,"""April""","""TMP-101""","""Prometne informacije 27.…","""2022-04-27""","""06:00""",0,0,"""1,2"""
2022,"""April""","""TMP-102""","""Prometne informacije 26.…","""2022-04-26""","""20:00""",0,0,"""2"""


In [7]:
dfOutputs.write_csv("../data/outputs.csv")